In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

from explore_tool import load_info,press,pressing,frame_to_delta,replay_to_xy,plot_trace_2d,plot_trace_3d,plot_trace_sequence,summary,\
    look_pressing,delta_equip,load_infos,show_pressing_in_time,show_pressing_in_time2

In [2]:
%%time
infos = load_infos('replay_int16/RB')
# 结果被迫优化读取方式以免卡爆内存

Wall time: 11.1 s


In [3]:
info_list = list(infos.values())
info = infos['th10_ud000c.rpy.json.pickle']
replay = info['stages'][0]['replay']

frame_matrix = np.array([[info['stages'][i]['frame'] for i in range(6)] for info in info_list])
frame_matrix.max(axis=0)

array([11520, 13158, 15880, 17332, 20524, 24119])

In [4]:
%%time 
for info in info_list:
    delta_equip(info)

Wall time: 4.73 s


In [5]:
frame_matrix.max(axis=0)//60 + 1

array([193, 220, 265, 289, 343, 402], dtype=int32)

In [6]:
len(info_list)

202

In [7]:
%%time
stage_list = []
choose_keys = ['pressing_shift','pressing_ctrl','pressing_x','move']
replay_number = len(info_list)
for i,max_second in enumerate(frame_matrix.max(axis=0)//60 + 1):
    stage_arr = np.zeros((replay_number,len(choose_keys),max_second))
    for j,info in enumerate(info_list):
        df = info['stages'][i]['replay'][choose_keys]
        df = df.groupby(df.index.map(lambda t:t//60)).sum()
        mat = df.as_matrix()
        stage_arr[j,:,:mat.shape[0]] = np.transpose(mat)
    stage_list.append(stage_arr)

Wall time: 12.5 s


In [8]:
for stage_arr in stage_list:
    print(stage_arr.shape)

(202, 4, 193)
(202, 4, 220)
(202, 4, 265)
(202, 4, 289)
(202, 4, 343)
(202, 4, 402)


In [9]:
t = 4 # talk mean

time_length_list = [
    [(80,3),(t,1),(47,10),(t,1)], # stage 1
    [(96,3),(t,1),(63,10),(t,1)], # stage 2
    [(60,3),(3,1),(20,1),(t,1),(20,5),(25,3),(t,1),(65,10),(t,1)], # stage 3
    [(145,10),(t,1),(100,20),(t,1)], # stage 4
    [(130,5),(t,1),(110,20),(t,1)], # stage 5
    [(58,3),(t,1),(240,60)] # stage 6
]


In [10]:
scen_4_meansd = []
for stage,time_length_mean_std in zip(stage_list,time_length_list):
    length = stage.shape[2]
    rights = np.array([item[0] for item in time_length_mean_std]).cumsum().tolist() + [length]
    
    four_musd = []
    
    for i,right in enumerate(rights):
        left = rights[i-1] if i >= 1 else 0
        
        musd = []
        
        for j in range(4):
            # we apply entire sub matrix instead of some array since we assume they're independ in sample and time
            mu = stage[:,j,left:right].mean() 
            sd = stage[:,j,left:right].std()
            musd.append([mu,sd])
        four_musd.append(musd)
    scen_4_meansd.append(four_musd)

In [11]:
latent_scen_4_meansd = []
border_std = [10,10,10,10,10]
for stage in scen_4_meansd:
    priori_list = []
    for period in stage:
        priori = [period[0][0], border_std[0]],[period[1][0],border_std[1]],[period[2][0],border_std[2]],\
                    [period[3][0],border_std[3]],[period[3][1],border_std[4]]
        priori_list.append(priori)
    latent_scen_4_meansd.append(priori_list)

In [13]:
%%time
import pymc3 as pm
from pymc3 import Normal,Poisson
from pymc3.math import switch,where

Wall time: 0 ns


这尼玛已经占了90%的内存了还跑个屁MCMC

In [17]:
arr = stage_list[0]

In [18]:
arr.size

155944

In [20]:
arr.astype('float16')

155944

In [23]:
len(arr.tobytes())

1247552

In [25]:
len(arr.astype('float16').tobytes())

311888

In [26]:
arr.astype('float16')

array([[[   0. ,    0. ,    0. , ...,    0. ,    0. ,    0. ],
        [   0. ,    0. ,    0. , ...,    0. ,    0. ,    0. ],
        [   0. ,    0. ,    0. , ...,    0. ,    0. ,    0. ],
        [   0. ,    0. ,  162. , ...,    0. ,    0. ,    0. ]],

       [[   0. ,    0. ,    2. , ...,    0. ,    0. ,    0. ],
        [   0. ,    0. ,    0. , ...,    0. ,    0. ,    0. ],
        [   0. ,    0. ,    0. , ...,    0. ,    0. ,    0. ],
        [   0. ,    0. ,  148.5, ...,    0. ,    0. ,    0. ]],

       [[  12. ,   60. ,   60. , ...,    0. ,    0. ,    0. ],
        [   0. ,    0. ,    0. , ...,    0. ,    0. ,    0. ],
        [   0. ,    0. ,    0. , ...,    0. ,    0. ,    0. ],
        [ 316.5,    2. ,    0. , ...,    0. ,    0. ,    0. ]],

       ..., 
       [[   4. ,   60. ,   60. , ...,    0. ,    0. ,    0. ],
        [   0. ,    0. ,    0. , ...,    0. ,    0. ,    0. ],
        [   0. ,    0. ,    0. , ...,    0. ,    0. ,    0. ],
        [ 489.5,   20. ,    0. , ...

In [27]:
arr

array([[[   0. ,    0. ,    0. , ...,    0. ,    0. ,    0. ],
        [   0. ,    0. ,    0. , ...,    0. ,    0. ,    0. ],
        [   0. ,    0. ,    0. , ...,    0. ,    0. ,    0. ],
        [   0. ,    0. ,  162. , ...,    0. ,    0. ,    0. ]],

       [[   0. ,    0. ,    2. , ...,    0. ,    0. ,    0. ],
        [   0. ,    0. ,    0. , ...,    0. ,    0. ,    0. ],
        [   0. ,    0. ,    0. , ...,    0. ,    0. ,    0. ],
        [   0. ,    0. ,  148.5, ...,    0. ,    0. ,    0. ]],

       [[  12. ,   60. ,   60. , ...,    0. ,    0. ,    0. ],
        [   0. ,    0. ,    0. , ...,    0. ,    0. ,    0. ],
        [   0. ,    0. ,    0. , ...,    0. ,    0. ,    0. ],
        [ 316.5,    2. ,    0. , ...,    0. ,    0. ,    0. ]],

       ..., 
       [[   4. ,   60. ,   60. , ...,    0. ,    0. ,    0. ],
        [   0. ,    0. ,    0. , ...,    0. ,    0. ,    0. ],
        [   0. ,    0. ,    0. , ...,    0. ,    0. ,    0. ],
        [ 489.5,   20. ,    0. , ...

看起来还行。。没办法了，全部转成float16节省内存，如果可能的话，起码也可以降成float32，因为theano刻意标出了这个特性。
然后中间过程就省掉了。

事实证明不能用float16只能用float32..

In [32]:
stage_list_float16 = [stage.astype('float16') for stage in stage_list]

In [33]:
stage_list_float16[0].dtype

dtype('float16')

In [34]:
stage_list_float16[0].shape

(202, 4, 193)

In [35]:
cache_dict = {
    'latent_scen_4_meansd': latent_scen_4_meansd,
    'time_length_list': time_length_list,
    'stage_list_float16': stage_list_float16
}

In [37]:
import pickle

In [38]:
with open('cache_dict_float16','wb') as f:
    pickle.dump(cache_dict, f)

# float 32 fold

In [12]:
import pickle
stage_list_float32 = [stage.astype('float32') for stage in stage_list]
cache_dict = {
    'latent_scen_4_meansd': latent_scen_4_meansd,
    'time_length_list': time_length_list,
    'stage_list_float32': stage_list_float32
}
with open('cache_dict_float32','wb') as f:
    pickle.dump(cache_dict, f)